In [ ]:
from neural_poisson.data.shapenet import ShapeNetCoreDataset
from pathlib import Path
import time

def measure_dataset(dataset, N=100):
    start_time = time.time()
    for i in range(N):
        batch = dataset[i]
    end_time = time.time()
    duration = ((end_time - start_time) / N) * 1000
    print(f"Time taken for 100 iterations (GPU): {duration:.6f} ms")

# define the paths
model_id = "1a04e3eab45ca15dd86060f189eb133"
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCoreTiny/02691156"
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

# load the dataset
dataset = ShapeNetCoreDataset(
    path=shapenet_path,
    segments=10,
    dist=2.0,
    fov=30.0,
    resolution=0.001,
    batch_size=10_000,
    max_empty_points=100_000,
    max_close_points=100_000,
    max_surface_points=100_000,
    vector_field_mode="nearest_neighbor"
)
measure_dataset(dataset)


dataset = ShapeNetCoreDataset(
    path=shapenet_path,
    segments=10,
    dist=2.0,
    fov=30.0,
    resolution=0.001,
    batch_size=10_000,
    max_empty_points=10_000,
    max_close_points=10_000,
    max_surface_points=10_000,
    vector_field_mode="nearest_neighbor"
)
measure_dataset(dataset)


In [ ]:
from neural_poisson.model.neural_poisson import NeuralPoisson
from neural_poisson.model.encoder import MLP

model = NeuralPoisson(encoder=MLP).to("cuda")

In [ ]:
import torch
import torch.utils.benchmark as benchmark

# Run the benchmark
t0 = benchmark.Timer(
    stmt="model(p)",  # Statement to benchmark
    setup="import torch",  # Setup code to run before the benchmark
    globals={"model": model, "p": p2},  # Ensure model and p2 are in the globals
)

# Print the results
print(t0.timeit(100))  # Run the statement 100 times and get the average time


In [ ]:
from tqdm.notebook import tqdm
N = 10_000
dataset.batch_size = 10_000
for i in tqdm(range(N)):
    batch = dataset[i]
    # model(batch["points_surface"])

In [ ]:
dataset.plot_point_cloud("points_surface")

In [ ]:
dataset.plot_normal_maps(False)

In [ ]:
from neural_poisson.model.neural_poisson import NeuralPoisson
import open3d as o3d

ckpt_path = "/home/borth/2d-gaussian-splatting/logs/2025-02-19/18-03-57/checkpoints/epoch_099.ckpt"
model = NeuralPoisson.load_from_checkpoint(ckpt_path)


In [ ]:
model.isolevel = 0.25
mesh = model.to_mesh()

In [ ]:
mesh.compute_vertex_normals()
o3d.visualization.draw_plotly([mesh])